# Simple RAG Application using Langchain

### Table of Contents

1. **Simple LLM App**
    * Load LLM
    * Create Prompt Teamplate
    * Merge Prompt Template & LLM to create Chain
2. **Build RAG App**
    * Load External Data
    * Generate Embeddings
    * Build Index in Vector Store (using Embeddings)
    * Create Retrieval Chain
    
    
### Installation

* **pip install langchain**

## 1. Simple LLM App

### 1.1 Load LLM

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2") # base_url = 'http://localhost:11434'

llm

Ollama()

In [2]:
response = llm.invoke("Do you know about Claude 3?")

print(response)

I'm not familiar with a person or product called "Claude 3." Could you please provide more context or information about who or what Claude 3 is? That way, I can better understand your question and give you an accurate answer.


### 1.2 Create Prompt Template

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Artificial Intelligence News Reporter."),
    ("user", "{query}")
])

### 1.3 Merge Prompt Template & LLM to Create LangChain Chain

In [4]:
llm_app = prompt | llm

llm_app

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an Artificial Intelligence News Reporter.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
| Ollama()

In [5]:
response = llm_app.invoke({"query": "Do you know about Claude 3?"})

print(response)


Ah, a question about the latest advancements in artificial intelligence! *excited tone* Indeed, I am aware of Claude 3, the latest and most advanced AI language model developed by Meta AI. It has been making waves in the AI community with its groundbreaking capabilities and unparalleled performance.

Claude 3 is a transformer-based language model that builds upon the successes of its predecessors, Claude 1 and Claude 2. It uses a novel architecture that combines the strengths of both models to create an AI that can generate human-like text with unprecedented accuracy and fluency. The model is trained on a vast amount of text data, including books, articles, and websites, allowing it to learn the intricacies of language and produce content that is not only grammatically correct but also contextually appropriate.

One of the most significant improvements of Claude 3 is its ability to generate text that is more diverse and creative. Unlike earlier models, which often produced repetitive 

## 2. Build RAG App

### 2.1 Load External Data

* **pip install beautifulsoup4**

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.anthropic.com/news/claude-3-family")

docs = loader.load()

In [7]:
urls = [
        "https://www.anthropic.com/news/releasing-claude-instant-1-2",
        "https://www.anthropic.com/news/claude-pro",
        "https://www.anthropic.com/news/claude-2",
        "https://www.anthropic.com/news/claude-2-1",
        "https://www.anthropic.com/news/claude-2-1-prompting",
        "https://www.anthropic.com/news/claude-3-family",
        "https://www.anthropic.com/claude"
       ] 

docs = []
for url in urls:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())
    
len(docs)

7

In [8]:
docs[0]

Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersAnnouncementsReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in our

### 2.2 Generate Embeddings

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings_llm = OllamaEmbeddings(model="llama2") # base_url = 'http://localhost:11434'

In [10]:
embeddings = embeddings_llm.embed_query("How are you?")

embeddings[:5]

[-0.4199983775615692,
 -0.34106385707855225,
 2.3779549598693848,
 0.24847780168056488,
 -0.13013841211795807]

In [11]:
type(embeddings), len(embeddings)

(list, 4096)

In [13]:
embeddings = embeddings_llm.embed_documents([
                                "Claude 3 is latest Conversational AI Model from Anthropic.",
                                "Gemini is latest Conversational AI Model from Google.",
                                "Llama-2 is latest Conversational AI Model from Meta.",
                                "Mixtral is latest Conversational AI Model from Mistral AI.",
                                "GPT-4 is latest Conversational AI Model from OpenAI."
                               ])

len(embeddings), type(embeddings), len(embeddings[0])

(5, list, 4096)

### 2.3 Build Index in Vector Store

* **pip install faiss-cpu**

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

len(documents)

12

In [15]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embeddings_llm)

vector_index

There are 3 functions to create index.

* **from_documents()**
* **from_embeddings()**
* **from_texts()**

In [16]:
retriever = vector_index.as_retriever()

relevant_docs = retriever.invoke({"input": "Do you know about Claude 3?"})

len(relevant_docs)

4

In [17]:
relevant_docs

[Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersAnnouncementsReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in ou

In [18]:
for doc in relevant_docs:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting
Title : Introducing Claude Pro \ Anthropic, Source: https://www.anthropic.com/news/claude-pro


### 2.4 Create Retrieval Chain

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context and your internal knowledge.
Give priority to context and if you are not sure then say you are not aware of topic:

<context>
{context}
</context>

Question: {input}
""")

#document_chain  = prompt | llm 
document_chain = create_stuff_documents_chain(llm, prompt)

response = document_chain.invoke({
    "input": "Do you know about Claude 3?",
    "context": [Document(page_content="Claude 3 is latest Conversational AI Model from Anthropic.")]
})

print(response)

Yes, I'm familiar with Claude 3. As an AI language model, I can provide information on the latest conversational AI models, including Claude 3 from Anthropic. According to my training data and knowledge, Claude 3 is a recent advancement in conversational AI technology that offers improved performance in natural language understanding and generation. It has been designed to better understand and respond to user input, enabling more engaging and human-like conversations.

Claude 3 leverages the power of deep learning algorithms and large datasets to learn patterns in language use and generate coherent and contextually appropriate responses. This allows it to converse with users in a more natural and intuitive way, making it an attractive option for businesses and organizations looking to integrate conversational AI into their products and services.

While I have knowledge of Claude 3, I cannot claim to be an expert on every topic or detail related to this model. If you have specific ques

In [20]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f800c06de80>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based on the provided context and your internal knowledge.\nGive priority to context and if you are not sure then say you are not aware of topic:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'))])
            | Ollama()
            | StrOutputParse

In [21]:
response = retrieval_chain.invoke({"input": "Do you know about Claude 3?"})

type(response)

dict

In [22]:
response.keys()

dict_keys(['input', 'context', 'answer'])